In [89]:
import numpy as np
import pandas as pd

import fastf1 as ff1
from fastf1 import plotting

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [90]:
ff1.Cache.enable_cache("cache")

In [91]:
race = ff1.get_session(2022, "Bahrain", "R")

laps = race.load_laps(with_telemetry=True)

/Users/juanpena/miniconda3/envs/f1/lib/python3.9/site-packages/fastf1/core.py:1093: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` instead.
  warnings.warn("`Session.load_laps` is deprecated and will be"
core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.2.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27'

In [92]:
gas, bot = "GAS", "BOT"

d_min, d_max = 4800, 5500

In [93]:
gas_l = laps.pick_driver(gas)
bot_l = laps.pick_driver(bot)

tel_gas = gas_l.pick_fastest().get_car_data().add_distance()
tel_bot = bot_l.pick_fastest().get_car_data().add_distance()

team_gas = gas_l.reset_index().loc[0, "Team"]
team_bot = bot_l.reset_index().loc[0, "Team"]

In [94]:
tel_gas.loc[tel_gas["Brake"] > 0, "CurrentAction"] = "Brake"
tel_gas.loc[tel_gas["Throttle"] == 100, "CurrentAction"] = "FullThrottle"
tel_gas.loc[(tel_gas["Brake"] == 0) & (tel_gas["Throttle"] < 100), "CurrentAction"] = "Cornering"

tel_bot.loc[tel_bot["Brake"] > 0, "CurrentAction"] = "Brake"
tel_bot.loc[tel_bot["Throttle"] == 100, "CurrentAction"] = "FullThrottle"
tel_bot.loc[(tel_bot["Brake"] == 0) & (tel_bot["Throttle"] < 100), "CurrentAction"] = "Cornering"

In [95]:
tel_gas["ActionId"] = (tel_gas["CurrentAction"] != tel_gas["CurrentAction"].shift(1)).cumsum()
tel_bot["ActionId"] = (tel_bot["CurrentAction"] != tel_bot["CurrentAction"].shift(1)).cumsum()

In [96]:
tel_gas

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,CurrentAction,ActionId
0,2022-03-20 15:59:26.718,11166,280,7,100,False,0,car,0 days 00:00:00.262000,0 days 01:58:26.701000,20.377778,FullThrottle,1
1,2022-03-20 15:59:27.158,11190,284,7,100,False,0,car,0 days 00:00:00.702000,0 days 01:58:27.141000,55.088889,FullThrottle,1
2,2022-03-20 15:59:27.518,11346,286,7,100,False,0,car,0 days 00:00:01.062000,0 days 01:58:27.501000,83.688889,FullThrottle,1
3,2022-03-20 15:59:27.878,11374,287,7,100,False,0,car,0 days 00:00:01.422000,0 days 01:58:27.861000,112.388889,FullThrottle,1
4,2022-03-20 15:59:28.038,11440,288,7,100,False,0,car,0 days 00:00:01.582000,0 days 01:58:28.021000,125.188889,FullThrottle,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,2022-03-20 16:01:02.438,10703,266,7,100,False,0,car,0 days 00:01:35.982000,0 days 02:00:02.421000,5259.826389,FullThrottle,27
350,2022-03-20 16:01:02.837,10856,272,7,100,False,0,car,0 days 00:01:36.381000,0 days 02:00:02.820000,5289.973056,FullThrottle,27
351,2022-03-20 16:01:03.157,11003,274,7,100,False,0,car,0 days 00:01:36.701000,0 days 02:00:03.140000,5314.328611,FullThrottle,27
352,2022-03-20 16:01:03.477,10852,278,7,100,False,0,car,0 days 00:01:37.021000,0 days 02:00:03.460000,5339.039722,FullThrottle,27


In [97]:
avg_gas = np.mean(tel_gas["Speed"].loc[
    (tel_gas["Distance"] >= d_min) & 
        (tel_gas["Distance"] <= d_max)
])

avg_bot = np.mean(tel_bot["Speed"].loc[
    (tel_bot["Distance"] >= d_min) & 
        (tel_bot["Distance"] <= d_max)
])

fastest = max(avg_gas, avg_bot)
slowest = min(avg_gas, avg_bot)

if fastest == avg_bot:
    speed_text = f"BOT {round(fastest - slowest, 2)} km/h faster"
else:
    speed_text = f"GAS {round(fastest - slowest, 2)} km/h faster"

In [98]:
tel_gas = tel_gas[["ActionId", "CurrentAction", "Distance"]].groupby(["ActionId", "CurrentAction"]).max("Distance").reset_index()
tel_bot = tel_bot[["ActionId", "CurrentAction", "Distance"]].groupby(["ActionId", "CurrentAction"]).max("Distance").reset_index()

tel_gas["Driver"] = "Gasly"
tel_bot["Driver"] = "Bottas"

In [99]:
tel_gas["DistanceDelta"] = tel_gas["Distance"] - tel_gas["Distance"].shift(1)
tel_gas.loc[0, "DistanceDelta"] = tel_gas.loc[0, "Distance"]

tel_bot["DistanceDelta"] = tel_bot["Distance"] - tel_bot["Distance"].shift(1)
tel_bot.loc[0, "DistanceDelta"] = tel_bot.loc[0, "Distance"]

In [100]:
all_act = tel_gas.append(tel_bot)

/var/folders/tv/r0kxfms90ks35z29rfdn387c0000gn/T/ipykernel_38123/3663558194.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_act = tel_gas.append(tel_bot)


In [109]:
plt.rcParams["figure.figsize"] = [20, 10]
plt.rcParams["figure.autolayout"] = True

telemetry_colors = {
    "Full Throttle": "green",
    "Cornering": "yellow",
    "Brake": "red",
}

fig, ax = plt.subplot(2)

ax[0].plot(tel_gas['Distance'], tel_gas['Speed'], label="GAS", color=ff1.plotting.team_color(tel_gas))
ax[0].plot(tel_bot['Distance'], tel_bot['Speed'], label="BOT", color=ff1.plotting.team_color(tel_bot))

# Speed difference
ax[0].text(d_min + 15,200, speed_text, fontsize = 15)

ax[0].set(ylabel='Speed')
ax[0].legend(loc="lower right")

for driver in ["GAS", "BOT"]:
    driver_actions = all_act.loc[all_act['Driver'] == driver]
    
    previous_action_end = 0
    for _, action in driver_actions.iterrows():
        ax[1].barh(
            [driver], 
            action['DistanceDelta'], 
            left=previous_action_end, 
            color=telemetry_colors[action['CurrentAction']]
        )
        
        previous_action_end = previous_action_end + action['DistanceDelta']
        
plt.xlabel('Distance')

plt.gca().invert_yaxis()

ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['left'].set_visible(False)

labels = list(telemetry_colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=telemetry_colors[label]) for label in labels]
ax[1].legend(handles, labels)

ax[0].set_xlim(d_min, d_max)
ax[1].set_xlim(d_min, d_max)

plt.savefig('2021_Silverstone_Q_HAM_vs_VER_Corner_Analysis.png', dpi=300)

ValueError: Single argument to subplot must be a three-digit integer, not 2

<Figure size 1440x720 with 0 Axes>